In [ ]:
# Insertar cuantos bloques de código y markdown consideren necesarios
import pandas as pd# Estructura y manipulación de datos
import seaborn as sns
from google.colab import files
import numpy as np # Programación vectorial
from seaborn import boxplot
from numpy import array
from scipy import stats

# Gráficos

import matplotlib.pyplot as plt # Visualización
from matplotlib import style


# Preprocesado y modelado
from scipy.stats import pearsonr
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
#import statsmodels.api as sm
#import statsmodels.formula.api as smf
# Configuración warnings
import warnings
warnings.filterwarnings('ignore')

# Configuración matplotlib
plt.rcParams['image.cmap'] = "bwr"
#plt.rcParams['figure.dpi'] = "100"
plt.rcParams['savefig.bbox'] = "tight"
style.use('ggplot') or plt.style.use('ggplot')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#rt ="/content/drive/MyDrive/Base_clientes_Monopoly THIS.xlsx"
rt="/content/drive/MyDrive/Base_clientes_Monopoly-0.xlsx"# poner la ruta de tu monopoli.xlsx que tienes en tu drive

df = pd.read_excel(rt)
df.head(5)

In [ ]:

#uploaded = files.upload()
#for fn in uploaded.keys():
#    name = fn
#df = pd.read_excel(name)  # Usa read_excel para leer archivos de Excel
#df.head(5)


Fase 2 Comprender los datos

Analisis exploratorio de datos :EDA



In [ ]:
df.head(10)

In [ ]:
import pandas as pd

# Asegurarte de que pandas muestre toda la información de las columnas
pd.set_option('display.max_columns', None)

# Mostrar toda la información de las variables (columnas)
df.info()


In [ ]:
#TIPO DE DATOS
df.info

In [ ]:
# dimensiones DEL CONJUNTO DE DATOS
df.shape()

DATOS FALTANTES

In [ ]:
df.isna().sum().sort_values(ascending=False)
#EN ORDEN DESCENDENTE

In [ ]:
#total nulos
df.isna().sum().sum()

Estadísticas descriptivas**

 **Variables numéricas**  Estadísticas descriptivas : la media, desviación estándar, mínimo, cuartiles y máximo para columnas numéricas.

In [ ]:
df.describe()

Medidas de tendencia central
(moda/mediana/media)

In [ ]:
# Seleccionar solo las columnas numéricas
numeric_df = df.select_dtypes(include=[np.number])

# Obtener medidas de tendencia central para todas las columnas numéricas
mean = numeric_df.mean()
median = numeric_df.median()
mode = numeric_df.mode().iloc[0]  # La primera fila contiene la moda

# Crear un DataFrame con las medidas
summary_df = pd.DataFrame({
    'Media': mean,
    'Mediana': median,
    'Moda': mode
})

# Mostrar la tabla con medidas de tendencia central
summary_df.T

Medidas de Dispersion


Rango: La diferencia entre el valor máximo y el valor mínimo.
Varianza: La media de las diferencias al cuadrado entre cada valor y la media.
Desviación Estándar: La raíz cuadrada de la varianza. Indica cuánto se dispersan los valores respecto a la media.
Coeficiente de Variación: La desviación estándar dividida por la media, expresada como porcentaje.

In [ ]:
import numpy as np
import pandas as pd

# Seleccionar solo las columnas numéricas
numeric_df = df.select_dtypes(include=[np.number])

# Obtener medidas de dispersión para todas las columnas numéricas
std_dev = numeric_df.std()  # Desviación estándar
variance = numeric_df.var()  # Varianza
range_ = numeric_df.max() - numeric_df.min()  # Rango (max - min)
iqr = numeric_df.quantile(0.75) - numeric_df.quantile(0.25)  # Rango intercuartílico (IQR)
mean = numeric_df.mean()  # Media

# Calcular el Coeficiente de Variación (CV)
cv = (std_dev / mean) * 100

# Crear un DataFrame con las medidas
dispersion_df = pd.DataFrame({
    'Desviación Estándar': std_dev,
    'Varianza': variance,
    'Rango': range_,
    'Rango Intercuartílico': iqr,
    'Coeficiente de Variación (%)': cv
})

# Mostrar la tabla con medidas de dispersión
dispersion_df.T


medidas de forma : kurtosis, asimetria

In [ ]:
import pandas as pd
from scipy.stats import kurtosis, skew

# Seleccionar solo las columnas numéricas
numeric_df = df.select_dtypes(include=[np.number])

# Calcular medidas de forma
skewness = numeric_df.apply(lambda x: skew(x.dropna()))
kurtosis_values = numeric_df.apply(lambda x: kurtosis(x.dropna()))

# Crear un DataFrame con las medidas de forma
shape_df = pd.DataFrame({
    'Asimetría (Skewness)': skewness,
    'Curtosis': kurtosis_values
})

# Mostrar la tabla con medidas de forma
shape_df.T

##**Analisis Distribucion de las variables Numericas**

Visualizamos las distribucion de las variable para identificar posibles sesgos, y observar características como la normalidad, la presencia de valores atípicos y la dispersión de los datos.Lo que nos ayuda a elegir el método de normalización de características mas optimo.
Identificar que variables tienen una distribución normal, se debe observar la media y la desviación estándar, así como considerar la simetría de los datos.

Simetría: La distribución es simétrica alrededor de la media.
Media y mediana iguales: La media y la mediana son aproximadamente iguales.

In [ ]:
numeric_columns = df.select_dtypes(include=[np.number]).columns
numeric_columns

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Obtener todas las columnas numéricas
numeric_columns = df.select_dtypes(include=[np.number]).columns

# Configurar el tamaño de las figuras y el número de variables por página
num_vars_per_page = 10  # Ajusta esto según el número de gráficos que quieras por página
num_pages = (len(numeric_columns) + num_vars_per_page - 1) // num_vars_per_page

for page in range(num_pages):
    plt.figure(figsize=(16, num_vars_per_page * 4))

    # Seleccionar variables para esta página
    start = page * num_vars_per_page
    end = min(start + num_vars_per_page, len(numeric_columns))
    page_vars = numeric_columns[start:end]
    # Crear un gráfico de distribución para cada variable en la página
    for i, var in enumerate(page_vars):
        plt.subplot(num_vars_per_page, 2, i + 1)  # Crear una cuadrícula con num_vars_per_page filas y 2 columnas
        sns.histplot(df[var].dropna(), kde=True, bins=30)
        plt.title(f'Distribución de {var}')
        plt.xlabel(var)
        plt.ylabel('Frecuencia')

    plt.tight_layout()
    plt.show()

In [ ]:
# Variable : Sexo
#Total hombres y mujeres

total_m = 0
total_h = 0
for cantidad in df['Sexo']:
    if cantidad == 'M':
        total_m += 1
    elif cantidad == 'H':
        total_h += 1
print(f" {total_m} mujeres  y {total_h} hombres")

In [ ]:
#Grafico de barras de Cantidad de cuentas por cliente

plt.figure(figsize=(8, 7))
sns.countplot(data=df, x='Cuentas')
plt.xlabel('Cantidad de cuentas por cliente')
plt.ylabel('Clientes')
plt.title('Grafico de barras de Cantidad de cuentas por cliente')
plt.show()

In [ ]:
#Graficos de analisis distribucion

variables_num = ['Renta', 'Edad', 'TC', 'Fac_T12', 'UsoL1_T12', 'CUPO_L1']
for variable in variables_num:
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    sns.histplot(data=df, x=variable, hue='target', bins=30, kde=True)
    plt.xlabel(variable)
    plt.ylabel('Frecuencia')
    plt.title(f'Distribución de {variable} para clientes que pagan y no')
    plt.tight_layout()
    plt.show()

In [ ]:
#Boxplot de edad
plt.figure(figsize=(10, 6))
sns.boxplot(y='Edad', data=df)
plt.title('Distribución de la Edad')
plt.show()

In [ ]:
#Scatter plot edad - renta: relacion entre 2 variables
plt.figure(figsize=(10, 6))
sns.scatterplot(x='Edad', y='Renta', data=df)
plt.title('Relación entre Edad y Renta')
plt.xlabel('Edad')
plt.ylabel('Renta')
plt.show()

In [ ]:
#Violin plot edad - sexo
plt.figure(figsize=(10, 6))
sns.violinplot(x='Sexo', y='Edad', data=df)
plt.title('Distribución de la Edad por Sexo')
plt.xlabel('Sexo')
plt.ylabel('Edad')
plt.show()

escalado de caracteristicas

Estandarización:
Ajusta los datos para que tengan una media de 0 y una desviación estándar de 1. Utiliza StandardScaler para las columnas con distribución normal.

La variable que tiende a distribucion normal es edad, para aplicar estandarizacion.

Normalización:
Ajusta los datos para que se encuentren en un rango específico, generalmente entre 0 y 1. Utiliza MinMaxScaler para las columnas con distribución no normal.

Normalizaremos las características para que las características que tienen no tiene una distribucion normal.

CORRELACION ⁉

Se analiza la correlación que existe en las variables entre si,  teniendo en cuenta que aunque exista correlación no necesariamente implica que exista causalidad.<br>
Se observa que existen correlaciones tanto positivas como negativas.

In [ ]:
#CORRELACION
corr = df.corr(numeric_only=True)
corr

In [ ]:
#Mapa de correlacion

columnas_heatmap = ['Subsegmento','Region','Edad','Renta','Antiguedad','Internauta','Adicional','Dualidad','Monoproducto','Ctacte','Consumo', 'Hipotecario','Debito','Cuentas','TC','CUPO_L1','CUPO_MX','CUPO_L2','Col_T12','ColL1TE_T12','EeccInt_T12','EeccNac_T12','Fac_T12','FacAI_T12','UsoL1_T12', 'UsoL2_T12','target']
df_heatmap = df[columnas_heatmap]
matriz_correlacion = df_heatmap.corr()
plt.figure(figsize=(20, 18))
sns.heatmap(matriz_correlacion, annot=True, cmap='coolwarm', linewidths=0.5)
plt.title('Mapa de Correlación')
plt.show()

In [ ]:
top_corr = corr['Renta'].sort_values(ascending=False)[1:16]
print(top_corr)

##**OUTLIERS**

In [ ]:
#Analisis de outliers con Z-score

numeric_cols = df.select_dtypes(include=[np.number]).columns
z_scores = np.abs(stats.zscore(df[numeric_cols]))
#Z-score > 3
outliers = np.where(z_scores > 3)
print("Filas con outliers:", outliers[0])
print("Columnas con outliers:", outliers[1])
outlier_rows = df.iloc[outliers[0]]
outlier_rows

In [ ]:
#ANALIS DE OUTLIERS CON IQR
#Obtener todas las columnas numericas
numeric_columns = df.select_dtypes(include=[np.number]).columns
for column in df[numeric_columns].columns:
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)

        IQR = Q3 - Q1

        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        outliers = (df[column] < lower_bound) | (df[column] > upper_bound)

        outliers_count = outliers.sum()
        outliers_percentage = outliers.mean() * 100

        print(f"Columna '{column}' tiene {outliers_count} outliers.")
        print(f"Porcentaje de outliers: {outliers_percentage:.2f}%")
        print("---")

total_outliers = sum(outliers.sum() for column in df.columns)
total_entries = len(df) * len(df.columns)
total_percentage = total_outliers / total_entries * 100

print(f"Porcentaje total de outliers en todo el dataset: {total_percentage:.2f}%")
print(f"Cantidad total de outliers en todo el dataset: {total_outliers}")


In [ ]:
#DIAGRAMA DE CAJAS DE TODAS LAS VARIABLES NUMERICAS
numeric_columns = df.select_dtypes(include=[np.number]).columns
numeric_columns
plt.figure(figsize=(10,25))
df.boxplot(column=['Subsegmento', 'Region', 'Edad', 'Renta', 'Antiguedad', 'CUPO_L1', 'CUPO_L2', 'CUPO_MX', 'Fac_T12','FacCN_T12','FacCCOT_T12', 'FacDebCom_T12', 'TxsDebCom_T12', 'FacDebAtm_T12', 'TxsDebAtm_T12', 'Col_T12', 'ColL1T0_T12', 'ColL1TE_T12', 'ColL2T0_T12', 'ColL2CC_T12', 'PagoNac_T12', 'EeccNac_T12','UsoL1_T12', 'UsoL2_T12', 'UsoLI_T12']
, vert=False)
plt.show()

## **Estadísticas descriptivas**
**Variables Categoricas**

- COUNT : Cantidad de datos en la Variable
- UNIQUE : Esto representa la cantidad de datos unicos en la columna.
- TOP : Es el valor mas frecuente de la columna
- FREQ : Este es el numero de veces que se repite el valor de TOP

In [ ]:
# Seleccionar solo las columnas categóricas
categorica_df = df.select_dtypes(include=['object', 'category'])

# Mostrar estadísticas descriptivas de las columnas categóricas
categorica_df.describe()


**Grafico de barra**

Se muestra la frecuencia de cada clase en la respectiva columna ,  permiten visualizar rápidamente qué clase son más comunes en cada variable categórica.
- Tambien permite ver el balanceo de las clases.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Seleccionar solo las columnas categóricas
categorica_df = df.select_dtypes(include=[object])

# Crear un gráfico de barras para cada columna categórica
for column in categorica_df.columns:
    plt.figure(figsize=(12, 6))

    # Contar la frecuencia de cada clase
    counts = categorica_df[column].value_counts()

    # Crear el gráfico de barras
    sns.barplot(x=counts.index, y=counts.values, palette='viridis')

    plt.title(f'Frecuencia de cada clase en la columna {column}')
    plt.xlabel('Clases')
    plt.ylabel('Frecuencia')
    plt.xticks(rotation=90)  # Rotar etiquetas del eje x si hay muchas clases
    plt.show()


Las clases de las variables : estan desbalanceadas, significa que una de las clases está mucho más frecuente que la otra. Esto puede afectar la capacidad del  modelo para aprender correctamente las relaciones entre las características y las etiquetas. Se pueden aplicar tecnicas para manejar estas clases, como SMOTE, oversampling, undersampling.

**Datos Faltantes**

Errores tipográficos en variables categóricas
En una variable categórica pueden aparecer sub-niveles.Se deben unificar

In [ ]:
# Seleccionar solo las columnas categóricas
categorica_df = df.select_dtypes(include=[object])
categorica_df.info()
#Separo las variables por tipo y reviso los datos para ver si se repiten o se ve algo anormal

for col in categorica_df:
  print(f'Columna {col}: {df[col].unique()}{df[col].nunique()} subniveles')

## **Analisis de las variables objetivo**


In [ ]:
df['target'].value_counts()

In [ ]:
df['target'].isnull().sum()

In [ ]:
#GRAFICO DE barras
plt.figure(figsize=(4, 4))
df['target'].value_counts().plot(kind='bar')
# Agrega etiquetas y título para hacer el gráfico más informativo.
plt.xlabel('target')
plt.ylabel('Frecuencia')
plt.title('Gráfico de Barras de target')
# Muestra el gráfico de barras en una ventana gráfica
plt.show()




In [ ]:
# Configurar el tamaño del gráfico
plt.figure(figsize=(4, 4))

# Graficar la distribución de frecuencias de la columna 'EeccNac_T07'
df_regresion['EeccNac_T07'].value_counts().plot(kind='bar')

# Agregar etiquetas y título al gráfico
plt.xlabel('EeccNac_T07')
plt.ylabel('Frecuencia')
plt.title('Gráfico de Barras de target')

# Mostrar el gráfico
plt.show()


In [ ]:
df_regresion['EeccNac_T07'].describe()



In [ ]:
#Esto eliminará todas las filas donde EeccNac_T07 tiene valores nulos.#
df_regresion_clean = df_regresion.dropna(subset=['EeccNac_T07'])


TARGET NUMERICO

In [ ]:
# NULOS DE LA VARIABLE TARGET NUMERICO
df['CUPO_MX'].isnull().sum()

In [ ]:
df.CUPO_MX.value_counts()

In [ ]:
df['CUPO_MX'].describe()

In [ ]:
df['CUPO_MX'].mode()

CORRELACION CON EL TARGET NUMERICO

In [ ]:
# Filtrar solo las columnas numéricas del DataFrame
df_numeric = df.select_dtypes(include=['float64', 'int64'])

# Calcular la matriz de correlación solo para las columnas numéricas
corr = df_numeric.corr()

# Seleccionar las 15 variables más correlacionadas con 'CUPO_MX', excluyendo a sí misma
top_corr = corr['CUPO_MX'].drop('CUPO_MX').sort_values(ascending=False).head(30)

# Mostrar las correlaciones
print(top_corr)


In [ ]:
#Boxplot de variables de interes
plt.figure(figsize=(10, 6))
sns.boxplot(y='CUPO_MX', data=df)
plt.title('Distribución de ...')
plt.show()

In [ ]:
#Mapa de correlacion de variables de interes
columnas_heatmap = ['NombreVar','NombreVar']
df_heatmap = df[columnas_heatmap]
matriz_correlacion = df_heatmap.corr()
plt.figure(figsize=(20, 18))
sns.heatmap(matriz_correlacion, annot=True, cmap='coolwarm', linewidths=0.5)
plt.title('Mapa de Correlación')
plt.show()

In [ ]:
_discriminado# se crea un nuevo df
df_discriminado = df[['Renta', 'CUPO_L1']]

In [ ]:
# tipo de dato del target
df['target'].dtype

## outliers

In [ ]:
#Obtener todas las columnas numericas
numeric_columns = df.select_dtypes(include=[np.number]).columns
for column in df[numeric_columns].columns:
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)

        IQR = Q3 - Q1

        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        outliers = (df[column] < lower_bound) | (df[column] > upper_bound)

        outliers_count = outliers.sum()
        outliers_percentage = outliers.mean() * 100

        print(f"Columna '{column}' tiene {outliers_count} outliers.")
        print(f"Porcentaje de outliers: {outliers_percentage:.2f}%")
        print("---")

total_outliers = sum(outliers.sum() for column in df.columns)
total_entries = len(df) * len(df.columns)
total_percentage = total_outliers / total_entries * 100

print(f"Porcentaje total de outliers en todo el dataset: {total_percentage:.2f}%")
print(f"Cantidad total de outliers en todo el dataset: {total_outliers}")

In [ ]:
# Seleccionar las columnas específicas
columnas_interes = ['Renta', 'EeccNac_T07']

# Inicializar el contador total de outliers
total_outliers = 0

# Iterar sobre las columnas de interés
for column in columnas_interes:
    Q1 = df[column].quantile(0.25)  # Primer cuartil (Q1)
    Q3 = df[column].quantile(0.75)  # Tercer cuartil (Q3)

    IQR = Q3 - Q1  # Rango intercuartílico (IQR)

    # Calcular límites inferior y superior para outliers
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Identificar outliers
    outliers = (df[column] < lower_bound) | (df[column] > upper_bound)

    # Contar los outliers en la columna actual
    outliers_count = outliers.sum()
    outliers_percentage = outliers.mean() * 100

    # Acumular el total de outliers
    total_outliers += outliers_count

    # Mostrar los resultados para la columna actual
    print(f"Columna '{column}' tiene {outliers_count} outliers.")
    print(f"Porcentaje de outliers: {outliers_percentage:.2f}%")
    print("---")

# Calcular el total de entradas
total_entries = len(df) * len(columnas_interes)

# Calcular el porcentaje total de outliers
total_percentage = total_outliers / total_entries * 100

# Mostrar los resultados generales
print(f"Porcentaje total de outliers en las columnas seleccionadas: {total_percentage:.2f}%")
print(f"Cantidad total de outliers en las columnas seleccionadas: {total_outliers}")


In [ ]:
plt.figure(figsize=(10,25))
df.boxplot(column=['Subsegmento', 'Region', 'Edad', 'Renta', 'Antiguedad', 'CUPO_L1', 'CUPO_L2', 'CUPO_MX', 'Fac_T12','FacCN_T12','FacCCOT_T12', 'FacDebCom_T12', 'TxsDebCom_T12', 'FacDebAtm_T12', 'TxsDebAtm_T12', 'Col_T12', 'ColL1T0_T12', 'ColL1TE_T12', 'ColL2T0_T12', 'ColL2CC_T12', 'PagoNac_T12', 'EeccNac_T12','UsoL1_T12', 'UsoL2_T12', 'UsoLI_T12']
, vert=False)
plt.show()

FASE 3 : PREPARACION DE LOS DATOS

**transformacion logaritmica**

se aplica cuando los datos están sesgados positivamente, es decir, cuando la distribución tiene una larga cola hacia la derecha (valores extremadamente grandes). Esta técnica se usa para reducir la asimetría y hacer que los datos se acerquen más a una distribución normal, para  mejorar la interpretación y los resultados de ciertos modelos

Indicadores de cuándo aplicar una transformación logarítmica:
Sesgo positivo: Si los datos tienen un sesgo positivo significativo (con más valores concentrados en el lado inferior de la distribución y una cola larga hacia la derecha), la transformación logarítmica puede ayudar a reducir este sesgo.

Valores extremadamente grandes (outliers): Si los datos contienen valores muy grandes que están distorsionando la escala de la variable, una transformación logarítmica puede comprimir estos valores y hacer que los outliers sean menos influyentes.

Distribución sesgada o con cola larga: Al observar la distribución de la variable, si ves una cola larga que se extiende hacia valores altos, esta es una señal clara de que la transformación logarítmica podría ser beneficiosa.

Heterocedasticidad: Si en un análisis de regresión los errores no son constantes (heterocedasticidad), la transformación logarítmica puede estabilizar la varianza.

Modelos que asumen normalidad:  como la regresión lineal, asumen que los datos tienen una distribución normal.

In [ ]:
import numpy as np

# Asegúrate de que los valores no sean cero o negativos antes de la transformación
df['Renta_log'] = np.log(df['Renta'].replace(0, np.nan)).fillna(0)
df['EeccNac_T07_log'] = np.log(df['EeccNac_T07'].replace(0, np.nan)).fillna(0)


Precauciones:
Valores negativos o cero: La transformación logarítmica solo es válida para valores mayores que cero. Si tienes valores cero o negativos en tus datos, puedes:

Eliminar esos valores si son pocos.
Transformar los valores negativos o cero añadiendo un valor constante a todos los datos. Por ejemplo, sumar una pequeña cantidad (como 1) a la variable para evitar tomar logaritmos de cero:

Reversibilidad: Si transformas los datos logarítmicamente, debes recordar que para interpretar los resultados o revertir la transformación, deberás aplicar la exponencial inversa (np.exp()).

Escalado de variables

In [ ]:
#Opciones de codigo para fase 3

from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, SimpleImputer, KNNImputer, LabelEncoder
from sklearn.impute import KNNImputer
from sklearn.decomposition import PCA



#Logaritmo natural (normalizacion)
df['NombreVar'] = np.log1p(df['NombreVar'])

#Elevacion al cuadrado (normalizacion)
df['NombreVar'] = df['NombreVar'] ** 2



#Min-Max Scaling (escalamiento)
scaler_min_max = MinMaxScaler()
df[['NombreVar', 'NombreVar2', 'NombreVar3']] = scaler_min_max.fit_transform(df[['NombreVar', 'NombreVar2', 'NombreVar3']])

#Estandarizacin (escalamiento)
scaler_standard = StandardScaler()
df[['NombreVar', 'NombreVar2', 'NombreVar3']] = scaler_standard.fit_transform(df[['NombreVar', 'NombreVar2', 'NombreVar3']])

#Robust Scaling (escalamiento)
scaler_robust = RobustScaler()
df[['NombreVar', 'NombreVar2']] = scaler_robust.fit_transform(df[['NombreVar', 'NombreVar2']])



#Imputacion por mediana
imputer_median = SimpleImputer(strategy='median')
df[['NombreVar', 'NombreVar2', 'NombreVar3']] = imputer_median.fit_transform(df[['NombreVar', 'NombreVar2', 'NombreVar3']])

#Imputacion por KNN
imputer_knn = KNNImputer(n_neighbors=5)
df[['NombreVar', 'NombreVar2']] = imputer_knn.fit_transform(df[['NombreVar', 'NombreVar2']])

#Imputacion de valores extremos
df.fillna(df.mode().iloc[0], inplace=True)

#Eliminacion de outliers basado en Z-Score
z_scores = np.abs(stats.zscore(df[['NombreVar', 'NombreVar2', 'NombreVar3']]))
df = df[(z_scores < 3).all(axis=1)]

#Eliminar la columna Unnamed: 574 (agregar las columnas que sea necesario)
df = df.drop(columns=['Unnamed: 574'], errors='ignore')

#Tratar outliers en variables (con los percentiles)
for col in ['NombreVar', 'NombreVar2']:
    q_low = df[col].quantile(0.01)
    q_high = df[col].quantile(0.99)
    df = df[(df[col] > q_low) & (df[col] < q_high)]



#Convertir los categoricos en numericos
label_encoder = LabelEncoder()
df['NombreVar'] = label_encoder.fit_transform(df['NombreVar'])
df['NombreVar2'] = label_encoder.fit_transform(df['NombreVar2'])



#Eliminar variables altamente correlacionas que son irrelevantes para nuestro target???



#Reduccion de dimensionalidad con PCA ??????
from sklearn.decomposition import PCA
#Var a las que le aplicaremos PCA para reducirle la dimensionalidda
columnas_pca = ['NombreVar', 'NombreVar2', 'NombreVar3']
#Escalar
scaler = StandardScaler()
X_escalado = scaler.fit_transform(df[columnas_pca])
#PCA
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_escalado)
#Crear nuevo df y añadirle las var con pca
df_pca = pd.DataFrame(X_pca, columns=['PC1', 'PC2'])
#Agregar columnas del df original al df nuevo que tiene var con pca
columnas_adicionales = ['NombreVar1', 'NombreVar2']
#Crear un df con todas las var que necesitams
df_adicionales = df[columnas_adicionales]
#Combinar todo
df_final = pd.concat([df_adicionales, df_pca], axis=1)
#Imprimir primeras filas
print(df_final.head())







###**1- Imputación simple: datos faltantes completamente aleatorios**

Se utiliza una única regla para llenar los valores faltantes, lo que lo hace simple y rápido, pero con riesgo de cambiar la distribución de los datos.

a- a.Imputación por la media o la mediana

Cuándo usarla: Cuando los datos faltantes son numéricos y la variable no tiene sesgo extremo. Ventaja: Simple y rápido de implementar. Desventaja: Puede alterar la distribución de los datos si hay muchos valores faltantes, ya que todos se reemplazan por el mismo valor, reduciendo la variabilidad.

In [ ]:
#a.Imputación por la media
#Si el valor de la variable es numérico continuo,imputar con media o mediana es común. Si hay outliers, la mediana es preferible.
#Si los valores de EeccNac_T07 son aproximadamente simétricos o no tienen demasiados valores atípicos, puedes imputar los nulos con la media de la variable.
mean_value = df_regresion['EeccNac_T07'].mean()
df_regresion['EeccNac_T07'].fillna(mean_value, inplace=True)
#distribución normal, pero ten cuidado si hay valores extremos que puedan afectar la media.

In [ ]:
#Imputar con la Mediana
#La mediana es una mejor opción cuando los datos tienen valores extremos o están sesgados, ya que es menos sensible a los valores atípicos.
median_value = df_regresion['EeccNac_T07'].median()
df_regresion['EeccNac_T07'].fillna(median_value, inplace=True)


In [ ]:
#b. Imputación por una constante**
df['Renta'].fillna(0, inplace=True)  # Reemplazar faltantes con una constante


In [ ]:
#Imputación por regresión**

#Cuando hay** correlación fuerte** entre las variables, y se puede estimar el valor de una variable faltante usando otras variables.
# Preserva la distribución de los datos mejor que la media o mediana.

from sklearn.linear_model import LinearRegression

# Datos donde no hay nulos
complete_data = df.dropna(subset=['Renta'])

# Modelar la renta en función de otras variables
X = complete_data[['Edad', 'Antiguedad']]
y = complete_data['Renta']
reg = LinearRegression().fit(X, y)

# Predecir los valores faltantes
missing_renta = df[df['Renta'].isna()]
df.loc[df['Renta'].isna(), 'Renta'] = reg.predict(missing_renta[['Edad', 'Antiguedad']])


In [ ]:
#d. Imputación con k-Nearest Neighbors (KNN) o Hot-Deck. Cuandono hay correlación entre las variables, pero hay vecinos cercanos que pueden servir como referencia para imputar los datos.
#El K-Nearest Neighbors Imputer es una técnica más avanzada que usa la similitud entre registros para imputar los valores faltantes. Este método puede ser útil si los datos tienen correlaciones entre múltiples variables.
#Ventaja: Mantiene patrones locales en los datos y es más preciso que la media o mediana.
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=5)
df_imputed = imputer.fit_transform(df[['Edad', 'Renta', 'Antiguedad']])



In [ ]:
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=5)
df_regresion[['EeccNac_T07']] = imputer.fit_transform(df_regresion[['EeccNac_T07']])


In [ ]:
# Imputar con la Moda:Si la variable tiene valores que son más categóricos o tienes una cantidad limitada de valores posibles, puedes imputar con la moda (el valor más frecuente)
mode_value = df_regresion['EeccNac_T07'].mode()[0]
df_regresion['EeccNac_T07'].fillna(mode_value, inplace=True)


## **Escalado de características**
###**Normalización:**

Ajusta los datos para que se encuentren en un rango específico, generalmente entre 0 y 1.
Utiliza MinMaxScaler para las columnas con distribución no normal.

###**Estandarización**:

Ajusta los datos para que tengan una media de 0 y una desviación estándar de 1.
Utiliza StandardScaler para las columnas con distribución normal.

FASE 4 MODELAMIENTO

In [ ]:
#Correlacion df_categorica
import seaborn as sns
import matplotlib.pyplot as plt

#Calcular la matriz de correlación
corr_matrix = df_categorica.drop('target', axis=1).corr()

#Graficar la matriz de correlación con un mapa de calor
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', linewidths=0.5)
plt.title('Matriz de Correlación de los Features')
plt.show()
#Correlación entre cada característica y el target
corr_with_target = df_categorica.corr()['target'].sort_values(ascending=False)
print(corr_with_target)
sns.pairplot(df_categorica, diag_kind='kde')
plt.show()

In [ ]:
#Clasificación KNN
#Librerías
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from imblearn.over_sampling import SMOTE  # Importar SMOTE
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

#Preparación de los datos
X = df_categorica.drop('target', axis=1)  #Características
y = df_categorica['target']                 #Variable objetivo

#División de los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

#Aplicar SMOTE en el conjunto de entrenamiento
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

#Normalización de los datos
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_resampled)
X_test_scaled = scaler.transform(X_test)

#Definición del modelo KNN
knn = KNeighborsClassifier(
    n_neighbors=3,
    weights='uniform',
    algorithm='auto',
    metric='minkowski',
    leaf_size=30,
    n_jobs=-1
)

#Validación cruzada
scores = cross_val_score(knn, X_train_scaled, y_train_resampled, cv=5)  # Usar y_train_resampled
print("Cross-validation scores:", scores)
print("Average cross-validation score:", np.mean(scores))
print("Standard deviation of cross-validation scores:", np.std(scores))

#Entrenamiento del modelo
knn.fit(X_train_scaled, y_train_resampled)

#Predicciones
y_pred = knn.predict(X_test_scaled)

#Evaluación del modelo
print("Matriz de confusión:")
print(confusion_matrix(y_test, y_pred))
print("\nReporte de clasificación:")
print(classification_report(y_test, y_pred))
print("\nPrecisión del modelo en el conjunto de prueba:", accuracy_score(y_test, y_pred))

#ROC
y_scores = knn.predict_proba(X_test_scaled)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test, y_scores)
roc_auc = auc(fpr, tpr)
plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, color='blue', label=f'Curva ROC (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='red', linestyle='--')  # Línea diagonal
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Tasa de Falsos Positivos')
plt.ylabel('Tasa de Verdaderos Positivos')
plt.title('Curva ROC')
plt.legend(loc='lower right')
plt.grid()
plt.show()


In [ ]:
#Clasificacion Random Forest
#Librerías
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

#Preparación de los datos
X = df_categorica.drop('target', axis=1)  #Características
y = df_categorica['target']               #Variable objetivo

#División de los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

#Aplicar SMOTE en el conjunto de entrenamiento
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

#Normalización de los datos
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_resampled)
X_test_scaled = scaler.transform(X_test)

#Definición del modelo Random Forest
rf = RandomForestClassifier(
    n_estimators=100,        # Número de árboles en el bosque
    max_features='sqrt',     # Número máximo de características para dividir un nodo
    max_depth=None,          # Sin límite de profundidad
    min_samples_split=2,     # Mínimo número de muestras para dividir un nodo
    min_samples_leaf=1,      # Mínimo número de muestras en un nodo hoja
    bootstrap=True,          # Usar muestras bootstrap
    random_state=42,         # Reproducibilidad
    n_jobs=-1                # Usar todos los núcleos disponibles
)

#Validación cruzada con búsqueda de hiperparámetros
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_features': ['sqrt', 'log2'],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)
grid_search.fit(X_train_scaled, y_train_resampled)

#Mejor estimador
best_rf = grid_search.best_estimator_
print(f"Best Hyperparameters: {grid_search.best_params_}")

#Entrenamiento del modelo con los mejores hiperparámetros
best_rf.fit(X_train_scaled, y_train_resampled)

#Predicciones
y_pred = best_rf.predict(X_test_scaled)

#Evaluación del modelo
print("Matriz de confusión:")
print(confusion_matrix(y_test, y_pred))
print("\nReporte de clasificación:")
print(classification_report(y_test, y_pred))
print("\nPrecisión del modelo en el conjunto de prueba:", accuracy_score(y_test, y_pred))

#ROC
y_scores = best_rf.predict_proba(X_test_scaled)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test, y_scores)
roc_auc = auc(fpr, tpr)
plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, color='blue', label=f'Curva ROC (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='red', linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Tasa de Falsos Positivos')
plt.ylabel('Tasa de Verdaderos Positivos')
plt.title('Curva ROC')
plt.legend(loc='lower right')
plt.grid()
plt.show()
